In [7]:
# helpers.py
import pickle
from time import sleep
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

def scroll_down(driver, time_sleep=1):
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Cuộn xuống dưới cùng của trang
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(time_sleep)

        # Chờ trang tải thêm nội dung
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Nếu chiều cao trang không đổi, tức là đã tải hết nội dung
        if new_height == last_height:
            break
        last_height = new_height  # Cập nhật chiều cao trang mới


def save_cookie(driver, cookie_name):
    with open(cookie_name, 'wb') as file:
        pickle.dump(driver.get_cookies(), file)

def load_cookie(driver, cookie_name):
    with open(cookie_name, 'rb') as file:
        for cookie in pickle.load(file):
            driver.add_cookie(cookie)

def log_error(text, filename="log_error.txt"):
    with open(filename, "a", encoding='utf-8') as f:
        f.write(text + "\n")

def get_data(soup, selector):
    try:
        return soup.select(selector)[0].text
    except IndexError:
        return None

def info_shop(soup):
    res = {'Đánh giá': None, 'tỉ lệ phản hồi': None, 'tham gia': None, 'Sản phẩm': None, 'thời gian phản hồi': None, 'Người theo dõi': None}
    res['shop_name'] = get_data(soup, ".fV3TIn")
    brand = get_data(soup, ".ZUZ1FO") or ('mall' if get_data(soup, ".official-shop-new-badge") == '' else 'normal')
    res['shop_brand'] = brand
    for i in soup.select('.NGzCXN > .YnZi6x'):
        try:
            res[i.label.text] = i.span.text
        except AttributeError:
            pass
    return res

def extract_feedback(soup):
    feedback_count = 0
    feedbacks = []
    feedback_list = soup.find_all('div', class_='shopee-product-rating')
    for feedback in feedback_list:
        user_name = feedback.find('a').text if feedback.find('a') else "Ẩn danh"
        star_rating = len(feedback.find_all('svg', class_='icon-rating-solid--active'))
        comment_div = feedback.find('div', style=lambda v: v and 'color: rgba(0, 0, 0, 0.87)' in v)
        comment = " ".join(comment_div.stripped_strings) if comment_div else None
        feedbacks.append({'user_name': user_name, 'star_rating': star_rating, 'comment': comment})
    
    
    return feedbacks
def scrape_all_feedback(driver, max_pages=10):
    all_feedbacks = []
    page_count = 0  # Biến đếm trang

    while page_count < max_pages:
        # Lấy mã nguồn của trang hiện tại
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Gọi hàm extract_feedback để lấy feedback từ trang hiện tại
        feedbacks = extract_feedback(soup)
        all_feedbacks.extend(feedbacks)

        try:
            # Tìm và click vào nút "next page"
            next_button = driver.find_element(By.XPATH, "//button[contains(@class, 'shopee-icon-button shopee-icon-button--right ')]")
            next_button.click()
            sleep(2)  # Đợi một chút để trang tải xong
            
            page_count += 1  # Tăng biến đếm trang
        except NoSuchElementException:
            # Nếu không tìm thấy nút "next page", thoát khỏi vòng lặp
            break

    return all_feedbacks

def extract_data(driver, soup):
    res = {
        'name_product': get_data(soup, ".WBVL_7 > span"),
        'price_origin': get_data(soup, ".qg2n76"),
        'price': get_data(soup, ".G27FPf"),
        'rate': get_data(soup, ".dQEiAI"),
        'review_count': get_data(soup, ".e2p50f > div[class='F9RHbS']"),
        'sale_count': get_data(soup, ".AcmPRb"),
        'like_count': get_data(soup, ".w2JMKY > .rhG6k7"),
        'fee_delivery': get_data(soup, ".PZGOkt"),
        'is_flash_sale': 1 if get_data(soup, ".x7M8PV") else 0,
        'quantity': get_data(soup, ".OaFP0p > div > div:nth-child(2)"),
        'descibe': get_data(soup, ".e8lZp3"),
        'detail': get_data(soup, ".Gf4Ro0")
    }
    shop = info_shop(soup.find(class_="page-product__shop"))
    res.update(shop)
    res['feedbacks'] = scrape_all_feedback(driver, max_pages=10)
    return res

In [11]:
# scraper.py
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
from config import config
from helpers import scroll_down, save_cookie, load_cookie, log_error, extract_data
from time import sleep
from bs4 import BeautifulSoup
from pymongo import MongoClient

# hàm setup mongodb
def setup_mongo():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["Shopee_data"]
    return db["Products"]

#  thiết lập Selenium driver
def setup_driver():
    chrome_options = Options()
    chrome_options.headless = False # Tắt chế độ headless (hiển thị trình duyệt)
    chrome_options.add_argument('disable-infobars')  # Tắt thanh thông tin
    chrome_options.add_argument('--disable-extensions') # Tắt các tiện ích mở rộng
    chrome_options.add_argument('--no-sandbox') # Tắt chế độ sandbox
    driver = uc.Chrome(options=chrome_options) # Khởi tạo trình điều khiển Chrome với các tùy chọn đã thiết lập
    driver.maximize_window()
    return driver

# Trích xuất dữ liệu từ một trang cụ thể và lưu dữ liệu vào MongoDB.
def scrape_page(driver, url_page, collection):
    driver.get(url_page)
    sleep(10)
    scroll_down(driver, 1.0)
    products = driver.find_elements(By.XPATH, '//li[@data-sqe="item"]')
    print(f"Tìm thấy {len(products)} sản phẩm trên {url_page}")
    
    if not products:
        print("Không tìm thấy sản phẩm nào!")
        return

    links = []
    for box in products:
        try:
            link_product = box.find_element(By.XPATH, './/a[@class="contents"]').get_attribute('href')
            link_image = box.find_element(By.XPATH, './/div[@class="relative z-0 w-full pt-full"]//img').get_attribute('src')
            links.append((link_product, link_image))
        except Exception as e:
            print(f"Lỗi khi lấy link từ một sản phẩm: {e}")
            continue

    for link, img in tqdm(links, desc="Scraping products", ncols=100, colour='green'):
        driver.get(link)
        sleep(8)
        scroll_down(driver, 1.0)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        data = extract_data(driver, soup)
        data.update({"link_image": img, "link_product": link})
        collection.insert_one(data)  # Insert data into MongoDB collection

# Hàm chính để trích xuất dữ liệu từ nhiều trang và lưu vào MongoDB.
def main():
    driver = setup_driver()
    driver.get(config['base_url'])
    load_cookie(driver, 'cookies.pkl')
    driver.refresh()

    collection = setup_mongo()  # Set up MongoDB collection
    for page in range(10):
        log_error(f'# Page {page}')
        url_page = f"{config['base_url']}/{config['brand_url']}?page={page}"
        scrape_page(driver, url_page, collection)

    driver.quit()

if __name__ == "__main__":
    main()


ImportError: cannot import name 'config' from 'config' (c:\Users\quynh\AppData\Local\Programs\Python\Python310\lib\site-packages\config\__init__.py)